<a href="https://colab.research.google.com/github/rsin46/pegasus-demo/blob/master/Pegasus_Combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training/converting to .PB

In [ ]:
MODELNAME = 'cnn_dailymail' #use to change max output len
BATCH_SIZE = 1
max_input_len = 2048

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/google-research/pegasus
!pip3 install tensorflow_transform
!pip3 install -e pegasus
%cd pegasus
!export PYTHONPATH=.

Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 736.00 KiB/s, done.
Resolving deltas: 100% (56/56), done.
     |████████████████████████████████| 368kB 8.2MB/s 
     |████████████████████████████████| 112kB 23.5MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 3.0MB 20.2MB/s 
     |████████████████████████████████| 8.3MB 32.4MB/s 
     |████████████████████████████████| 63.8MB 125kB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 153kB 55.4MB/s 
     |████████████████████████████████| 829kB 43.4MB/s 
     |████████████████████████████████| 1.4MB 47.5MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 122kB 44.8MB/s 
     |████████████████████████████████| 235

Obtaining file:///content/pegasus
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 1.1MB 13.4MB/s 
     |████████████████████████████████| 8.6MB 19.3MB/s 
     |████████████████████████████████| 3.5MB 50.2MB/s 
     |████████████████████████████████| 1.4MB 55.5MB/s 
     |████████████████████████████████| 411.0MB 40kB/s 
     |████████████████████████████████| 110.5MB 41kB/s 
     |████████████████████████████████| 983kB 49.2MB/s 
     |████████████████████████████████| 307kB 55.4MB/s 
     |████████████████████████████████| 358kB 53.6MB/s 
     |████████████████████████████████| 184kB 53.7MB/s 
     |████████████████████████████████| 368kB 49.9MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 655kB 49.2MB/s 
     |████████████████████████████████| 5.3MB 45.7MB/s 
     |████████████████████████████████| 512kB 54.0MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |█████████████

## Training fresh data from checkpoint

### Base model (no finetuning)

In [ ]:
%cd /content/pegasus/

/content/pegasus


In [ ]:
!mkdir -p ckpt/pegasus_ckpt
!gsutil -m rsync gs://pegasus_ckpt ckpt/pegasus_ckpt/

Building synchronization state...
Starting synchronization...
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
Copying gs://pegasus_ckpt/checkpoint...
Copying gs://pegasus_ckpt/model.ckpt-1500000.data-00000-of-00001...
| [6/6 files][  2.3 GiB/  2.3 GiB] 100% Done  41.8 MiB/s ETA 00:00:00           
Operation completed over 6 objects/2.3 GiB.                                      


## Converting trained model to .pb (vary batch size)

In [ ]:
%cd /content/pegasus

/content/pegasus


In [ ]:
!gsutil cp -r gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model /content/pegasus/ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model

Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


In [ ]:
!cp -r '/content/gdrive/My Drive/Models/arxiv/' '/content/pegasus/ckpt/pegasus_ckpt/'

cp: cannot stat '/content/gdrive/My Drive/Models/arxiv/': No such file or directory


In [ ]:
# max_target_len = 64 # Doesn't do anything, use MODELNAME and public_params defs

import itertools
import os
import time

from absl import logging
from pegasus.data import infeed
from pegasus.params import all_params  # pylint: disable=unused-import
from pegasus.params import estimator_utils
from pegasus.params import registry
import tensorflow as tf
from pegasus.eval import text_eval
from pegasus.ops import public_parsing_ops
import pandas as pd
from random import choice
from tensorflow.python.estimator.export import export
tf.enable_eager_execution()
# import tensorflow_transform as tft
data_name = MODELNAME
import tensorflow_transform as tft

master = ""
# model_dir = "./ckpt/pegasus_ckpt/%s"%data_name
model_dir = "./ckpt/pegasus_ckpt/"
use_tpu = False
iterations_per_loop = 1000
num_shards = 1
param_overrides = ("vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model"
",batch_size={0},beam_size=5,beam_alpha=0.6,max_input_len={1}".format(BATCH_SIZE, max_input_len))

eval_dir = os.path.dirname(model_dir)
checkpoint_path = model_dir
checkpoint_path = tf.train.latest_checkpoint(checkpoint_path )
params = registry.get_params('%s_transformer'%data_name)(param_overrides)
pattern = params.dev_pattern
input_fn = infeed.get_input_fn(params.parser, pattern,
                                    tf.estimator.ModeKeys.PREDICT)
parser, shapes = params.parser(mode=tf.estimator.ModeKeys.PREDICT)
RAW_DATA_FEATURE_SPEC = dict([("inputs", tf.io.FixedLenFeature(shapes['inputs'], tf.int64)), 
                              ('targets', tf.io.FixedLenFeature(shapes['targets'], tf.string))])

raw_feature_spec = RAW_DATA_FEATURE_SPEC.copy()
raw_feature_spec.pop('targets')
# raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
#         raw_feature_spec, default_batch_size=0)
def serving_input_fn():
    raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec, default_batch_size=BATCH_SIZE)
    serving_input_receiver = raw_input_fn()

    raw_features = serving_input_receiver.features
    return tf.estimator.export.ServingInputReceiver(
        raw_features, serving_input_receiver.receiver_tensors)

print(tf.executing_eagerly())
estimator = estimator_utils.create_estimator(master, 
                                            model_dir,
                                            use_tpu,
                                            iterations_per_loop,
                                            num_shards, params, include_features_in_predictions=False)

output = estimator.export_saved_model(
    "model/", serving_input_fn
)


True

INFO:tensorflow:Using config: {'_model_dir': './ckpt/pegasus_ckpt/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc54ecf2b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infe

In [ ]:
print(shapes)

{'inputs': [2048], 'targets': [128]}


In [ ]:
dir = '/content/pegasus/' + output.decode("utf-8")
!ls -l $dir/variables/

total 2220748
-rw-r--r-- 1 root root 2274013192 Aug 25 04:29 variables.data-00000-of-00001
-rw-r--r-- 1 root root      21694 Aug 25 04:29 variables.index


In [ ]:
!cp -rv $dir '/content/gdrive/My Drive/Models/'

'/content/pegasus/model/1598329714' -> '/content/gdrive/My Drive/Models/1598329714'
'/content/pegasus/model/1598329714/variables' -> '/content/gdrive/My Drive/Models/1598329714/variables'
'/content/pegasus/model/1598329714/variables/variables.data-00000-of-00001' -> '/content/gdrive/My Drive/Models/1598329714/variables/variables.data-00000-of-00001'
'/content/pegasus/model/1598329714/variables/variables.index' -> '/content/gdrive/My Drive/Models/1598329714/variables/variables.index'
'/content/pegasus/model/1598329714/saved_model.pb' -> '/content/gdrive/My Drive/Models/1598329714/saved_model.pb'


In [ ]:
raise Exception("Model copied to {}, Factory Reset Runtime to predict".format(dir))

# Compressing saved model

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


TIMEOUT: ignored

In [ ]:
MODEL_DIR = '1598319160'
source = '/content/gdrive/My Drive/Models/{}'.format(MODEL_DIR)

In [ ]:
!saved_model_cli show --dir=${source} --all

# Use saved model to predict 
**NOTE: Need to factory reset runtime before commencing**
Ctrl+F10

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# MODEL: gigaword for gigaword
#        cnn_dailymail for CNN Dailymail

# Aeslc
# 1597317576 - Batch of 8
# 1597369292 - Batch of 1
# 1597378344 - Batch of 16

# Base (params aeslc)
# 1597539174 - 4,512,32 4.28GB
# 1597541945 - 8,1024,32 14.99GB T4 DELETED

# Base (params CNN)
# 1597558764 - 8,1024,128 13.91GB T4. Takes too long DELETED

# Base (params xsum)
# 1597564312 - 4,1024,64 8.14GB K80
# 1598146584 - 8,512,64 8.27GB 
# 1598153319 - 1,1024,64 4.38GB Tesla P100
# 1598163012 - 1,4096,64 8.27GB Tesla T4. DELETE - results are garbage
# 1598173286 - 1,2048,64 4.3GB T4

# MODELNAME = 'xsum'

# 1598319160 - 1,1024,128 4.3GB T4
# 1598329714 - 1,2048,128 4.15GB K80

MODEL_DIR = '1598319160'
BATCH_SIZE = 1
INPUT_SHAPE = 1024
# TEXT = 'thermo.txt'
# TEXT = '1598319160_thermo_condensed.txt'
# TEXT = 'laws.txt'
TEXT = '1598319160_laws_condensed_condensed.txt'

In [ ]:
%cd /content/

/content


In [ ]:
!git clone http://github.com/rsin46/pegasus-demo.git
!git clone http://github.com/google-research/pegasus.git

Cloning into 'pegasus-demo'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 67 (delta 29), reused 48 (delta 16), pack-reused 0
Unpacking objects: 100% (67/67), done.
Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 7.06 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
!mkdir /content/pegasus-demo/model
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.9MB/s 


In [ ]:
# if MODELNAME == 'gigaword':
#   !cp '/content/gdrive/My Drive/Models/gigaword.tar.xz' '/content/pegasus-demo/model/gigaword.tar.xz'
#   !tar -xvf '/content/pegasus-demo/model/gigaword.tar.xz' -C '/content/pegasus-demo/model/' 
# elif MODELNAME == 'cnn_dailymail':
#   !cp '/content/gdrive/My Drive/Models/cnn_dailymail.xz' '/content/pegasus-demo/model/cnn_dailymail.xz'
#   !tar -xvf '/content/pegasus-demo/model/cnn_dailymail.xz' -C '/content/pegasus-demo/model/'
# else:

# source = '/content/gdrive/My\ Drive/Models/{}'.format(MODEL_DIR)
# !rsync -rv $source '/content/pegasus-demo/model/'

source = '/content/gdrive/My Drive/Models/{}'.format(MODEL_DIR)

In [ ]:
%cd /content/pegasus-demo
# Initialising variables

import datetime
current_time = datetime.datetime.now()
current_str = current_time.strftime("%d-%m_%H-%M")

OUT_TEXT = f'/content/gdrive/My Drive/Models/{current_str}_{TEXT}_{MODEL_DIR}.txt'
print(OUT_TEXT)

def append(line):
    with open(OUT_TEXT, 'a', buffering=1) as redf:
        redf.write(line)

/content/pegasus-demo
/content/gdrive/My Drive/Models/31-08_01-49_1598319160_laws_condensed_condensed.txt_1598319160.txt


In [ ]:
import text_eval
import public_parsing_ops
import tensorflow as tf
import numpy as np
import datetime
import time

_SPM_VOCAB = 'ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece",
                                                 _SPM_VOCAB)

# shapes = {
#     'cnn_dailymail': (1024, 128),
#     'gigaword':(128, 32),
#     'aeslc':(512, 32),
#     'arxiv':(512, 64)
# }

# model_dir = {
#     'cnn_dailymail': 'model/1595037231/',
#     'gigaword': 'model/1595095633/',
#     'aeslc': 'model/{}'.format(MODEL_DIR),
#     'arxiv': 'model/{}'.format(MODEL_DIR)
# }

model_dir = 'model/{}'.format(MODEL_DIR)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

imported = tf.saved_model.load(source, tags='serve')


text = open(TEXT, "r", encoding="utf-8").read()

shape = INPUT_SHAPE
# shape,_ = shapes[MODELNAME]

input_ids_raw = encoder.encode(text)
input_ids_split = list(zip(*([iter(input_ids_raw)]*shape)))

# TODO split by paragraph bundles
batch_list = list(chunks(input_ids_split,BATCH_SIZE))

out_file = open(OUT_TEXT,"a")
current_time = datetime.datetime.now()
append(current_time.strftime("%d-%b-%Y (%H:%M:%S.%f)") + '\n')

for section_id, batch in enumerate(batch_list):
  start = time.time()
  
  input_list = []

  for input_ids in batch:
    inputs = np.zeros(shape)
    idx = len(input_ids)
    if idx>shape: idx =shape

    inputs[:idx] = input_ids[:idx]

    example = tf.train.Example()

    example.features.feature["inputs"].int64_list.value.extend(inputs.astype(int))

    input_list.append(tf.constant([example.SerializeToString()]))
  
  # Pad if the last list doesn't fit into a batch
  if len(input_list) < BATCH_SIZE:
    for i in range(0,BATCH_SIZE - len(input_list)):
      input_list.append(input_list[-1])

  output = imported.signatures['serving_default'](examples=tf.concat(input_list, axis=0))
  prediction = text_eval.ids2str(encoder, output['outputs'].numpy(), None)
  end = time.time()
  duration = end - start
  print(f"Section {section_id} --- {duration}\nPREDICTION >> ", prediction)
  append(f'Section {section_id} --- {duration}s ...\n{prediction}\n\n')


Section 0 --- 12.75844669342041
PREDICTION >>  For instance, we feel anger at person X, whereas in fact the true source of this may be envy; below the level of conscious awareness we feel inferior in relation to X and want something he or she has. For instance, we feel anger at person X, whereas in fact the true source of this may be envy; below the level of conscious awareness we feel inferior in relation to X and want something he or she has.
Section 1 --- 8.953211784362793
PREDICTION >>  You can also try a simple experiment: Approach people you are meeting for the first time, or only know peripherally, with various positive thoughts—“I like them,” “They seem smart,” et cetera.. You can also try a simple experiment: Approach people you are meeting for the first time, or only know peripherally, with various positive thoughts—“I like them,” “They seem smart,” et cetera..


In [ ]:
infer = imported.signatures["serving_default"]
print(infer.inputs)
print(infer.output_shapes)

[<tf.Tensor 'input_example_tensor:0' shape=(1,) dtype=string>, <tf.Tensor 'global_step:0' shape=() dtype=resource>]
{'outputs': TensorShape([1, 128])}


# GPU type

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("GPU type: " + gpu.name)
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=b26f0c098218be4cb7266fddbb0f12a6267aec1a75d78cad66eec4199a8e3832
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
GPU type: Tesla P100-PCIE-16GB
Gen RAM Free: 7.0 GB  | Proc size: 6.2 GB
GPU RAM Free: 11781MB | Used: 4499MB | Util  28% | Total 16280MB


# Appendix

### Fine tuned model (change finetuned model name)

In [ ]:
!mkdir ckpt
!gsutil cp -r gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model
!gsutil -m cp -r gs://pegasus_ckpt/arxiv ckpt/pegasus_ckpt/

mkdir: cannot create directory ‘ckpt’: File exists
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.data-00000-of-00001...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.index...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.meta...


In [ ]:
!python3 pegasus/bin/train.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/arxiv/model.ckpt-340000 \
--model_dir=ckpt/pegasus_ckpt/arxiv --save_checkpoints_steps=100

In [ ]:
!cp -rv ckpt/pegasus_ckpt/arxiv/ /content/gdrive/My\ Drive/

## Analyse graph

In [ ]:
output = infer(tf.constant([example.SerializeToString()]))
text_eval.ids2str(encoder, output['outputs'].numpy(), None)

In [ ]:
input_serial = tf.concat([tf.constant([example.SerializeToString()]),tf.constant([example.SerializeToString()])],axis=0)

In [ ]:
input_serial

In [ ]:
%load_ext tensorboard

In [ ]:
dir = model_dir[MODELNAME]
!python import_pb_to_tensorboard.py --model_dir $dir --log_dir /tmp/tensorflow_logdir

In [ ]:
%tensorboard --logdir=/tmp/tensorflow_logdir